# Preparación del entorno

# Prueba con SQL

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:////content/dmeyf.db

In [ ]:
%%sql
create or replace table competencia_02 as 
select 
    *
from read_csv_auto('D:\Equipo\Documentos\Posgrado\2C23\datasets\competencia_02_crudo.csv')

## Queries Básicas

In [ ]:
%sql select * from competencia_02 limit 5

In [ ]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_02
group by foto_mes

In [ ]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_02
), clientes as (
    select distinct numero_de_cliente from competencia_02
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , null as clase_ternaria -- AGREGAR LÓGICA
    from todo t
    left join competencia_02 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1

In [ ]:
%%sql
alter table competencia_02 add column clase_ternaria VARCHAR(10)

In [ ]:
%%sql
update competencia_02
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_02.numero_de_cliente = targets.numero_de_cliente and competencia_02.foto_mes = targets.foto_mes;

In [ ]:
%%sql
copy competencia_02 to '/content/competencia_02.csv.gz' (FORMAT CSV, HEADER)